# Regional frequency analyses

In [ ]:
import matplotlib.pyplot as plt
import xdatasets
from lmoments3.distr import KappaGen
from sklearn.cluster import HDBSCAN, OPTICS, AgglomerativeClustering

import xhydro as xh
import xhydro.frequency_analysis as xhfa
import xhydro.gis as xhgis

This notebook will demonstrate how to use the `xHydro` library to perform regional frequency analyses on a dataset of streamflow data. Since the initial steps for regional frequency analysis are the same as those for local frequency analysis, users are encouraged to refer to the [Local frequency analysis](local_frequency_analysis.ipynb) notebook for an overview.

For this example, we will use the same dataset of hydrometric gauges covering parts of southern Quebec, ensuring continuity with the previous analysis while expanding to a regional scale. However, as regional analyses rely on having access to multiple sources of data, we will extract streamflow for 15 stations.

In [ ]:
ds = (
    xdatasets.Query(
        **{
            "datasets": {
                "deh": {
                    "id": ["02*"],
                    "regulated": ["Natural"],
                    "variables": ["streamflow"],
                }
            },
            "time": {"start": "1970-01-01", "minimum_duration": (30 * 365, "d")},
        }
    )
    .data.squeeze()
    .load()
)

# This dataset lacks some attributes, so let's add them.
ds = ds.rename({"streamflow": "q"})
ds["id"].attrs["cf_role"] = "timeseries_id"
ds["q"].attrs = {
    "long_name": "Streamflow",
    "units": "m3 s-1",
    "standard_name": "water_volume_transport_in_river_channel",
    "cell_methods": "time: mean",
}

# Clean some of the coordinates that are not needed for this example
ds = ds.drop_vars([c for c in ds.coords if c not in ["id", "time", "name"]])

timeargs = {
    "annual": {},
}

ds_4fa = xh.indicators.get_yearly_op(
    ds, op="max", timeargs=timeargs, missing="pct", missing_options={"tolerance": 0.15}
)
ds_4fa

## Explanatory variables

In regional frequency analyses, explanatory variables are used to help explain the spatial variation in hydrological extremes across different locations. These variables can include factors such as catchment area, elevation, precipitation, land use, and soil type, among others. By incorporating explanatory variables, we can account for the influence of geographic and environmental characteristics on extreme events, allowing for more accurate regional predictions.

### a) Extraction of watershed characteristics using `xhydro.gis`

In this example, we'll use catchment properties as our primary explanatory variables. However, other variables, such as climatological averages or land use data, can also be incorporated depending on the analysis requirements. For detailed steps on how to extract and work with these data, refer to the [GIS operations](gis.ipynb) notebook.


In [ ]:
gdf = xdatasets.Query(
    **{
        "datasets": {
            "deh_polygons": {
                "id": ["02*"],
                "regulated": ["Natural"],
                "variables": ["streamflow"],
            }
        },
        "time": {"start": "1970-01-01", "minimum_duration": (30 * 365, "d")},
    }
).data.reset_index()

dswp = xhgis.watershed_properties(
    gdf[["Station", "geometry"]], unique_id="Station", output_format="xarray"
)
cent = dswp["centroid"].to_numpy()
lon = [ele[0] for ele in cent]
lat = [ele[1] for ele in cent]
dswp = dswp.assign(lon=("Station", lon))
dswp = dswp.assign(lat=("Station", lat))
dswp = dswp.drop("centroid")
dswp

### b) Principal component analysis

After acquiring the explanatory variables, the next step is to process them using Principal Component Analysis (PCA) to reduce the dimensionality of the dataset. PCA helps simplify the dataset by transforming the original variables into a smaller set of uncorrelated components while retaining most of the variation in the data. This is accomplished using the function `xhydro.frequency_analysis.regional.fit_pca`.


In [ ]:
help(xhfa.regional.fit_pca)

In [ ]:
data_pca, pca = xhfa.regional.fit_pca(dswp, n_components=3)
data_pca

The results show that the correlation between the components is close to 0, indicating that the first three components are sufficiently independent. This suggests that these components can be effectively used for the remainder of our analysis, as they capture most of the variation in the explanatory variables without significant overlap or multicollinearity.


In [ ]:
data_pca.to_dataframe(name="value").reset_index().pivot(
    index="Station", columns="components"
).corr()

### c) Clustering

The results from the PCA can be used to group the stations into clusters based on their similarities in the principal components. Clustering helps identify regions with similar characteristics, enabling more targeted and accurate regional frequency analyses. This step is accomplished using `xhydro.frequency_analysis.regional.get_group_from_fit`, which supports clustering methods from `sklearn.cluster`. In this example, we will use `AgglomerativeClustering` to form 3 clusters based on the PCA results.


In [ ]:
help(xhfa.regional.get_group_from_fit)

In [ ]:
groups = xhfa.regional.get_group_from_fit(
    AgglomerativeClustering, {"n_clusters": 3}, data_pca
)
groups

In [ ]:
ax = plt.subplot(1, 1, 1)
gdf[gdf["Station"].isin(groups[0])].plot(ax=ax, color="red")
gdf[gdf["Station"].isin(groups[1])].plot(ax=ax, color="green")
gdf[gdf["Station"].isin(groups[2])].plot(ax=ax, color="blue")

## Regional Frequency Analysis

**Hosking and Wallis** introduced a method for regional frequency analysis based on L-moments, which are particularly useful for analyzing extreme events across different regions. Here’s a brief overview of the key concepts:

1. **L-Moments**: L-moments are statistical measures that are derived from linear combinations of order statistics. They offer a more robust alternative to traditional moments (like mean and variance), especially in the presence of outliers or when dealing with small sample sizes. L-moments are widely used in hydrological studies for estimating the characteristics of extreme values, as they provide more stable and reliable estimates.

2. **Regional Frequency Analysis**: This technique involves pooling data from multiple sites or regions to estimate the frequency distribution of extreme events, such as floods or droughts. The approach allows for the determination of common statistical parameters across different locations, enabling a more generalized and regional perspective on extreme event behavior. Hosking and Wallis's method focuses on fitting a regional frequency distribution and assessing how well it captures the characteristics of the data.

3. **Regional L-Moments**: These are L-moments calculated from pooled data across multiple sites within a region. By applying regional L-moments, we can estimate the parameters of regional frequency distributions and better understand the occurrence of extreme events in different parts of the region. This method improves the accuracy and robustness of predictions for extreme events, particularly when data from multiple sites are available.

Let's start by calculating the L-moments for each station using `xhydro.frequency_analysis.regional.calc_moments`.


In [ ]:
help(xhfa.regional.calc_moments)

In [ ]:
ds_moments = xhfa.regional.calc_moments(ds_4fa)
ds_moments

To perform regional frequency analysis, we need to reshape our datasets of annual maximums and L-moments based on the groupings identified by the clustering algorithm. Since there is no standardized naming convention for this new dimension, `xHydro` uses the name `group_id` for the cluster groupings. This reshaping process is handled by the function `xhydro.frequency_analysis.regional.group_ds`, which organizes the data according to the cluster assignments and prepares it for further analysis.


In [ ]:
help(xhfa.regional.group_ds)

In [ ]:
ds_groups = xhfa.regional.group_ds(ds_4fa, groups=groups)
ds_moments_groups = xhfa.regional.group_ds(ds_moments, groups=groups)

We now need to compute two important scores for evaluating the regional frequency analysis: the *H-Score* and the *Z-Score*. The H-Score measures the homogeneity of data across different sites or regions relative to the regional model. It helps determine how well the data from multiple sites align with the regional distribution. The Z-Score quantifies the discrepancy between the observed and expected values, standardized by their variability. It helps determine how well the theoretical distribution (based on the regional model) fits the observed data.

- **H-Score (Homogeneity Score)**

  - **H < 1: Homogeneous**  
  This indicates that the data from different sites are quite similar and align well with the regional model, suggesting that the regional model is suitable for the entire region.

  - **1 ≤ H < 2: Maybe Homogeneous**  
  This range suggests some heterogeneity, meaning the data might still generally fit the regional model, but there are some variations that the model does not fully capture.

  - **H ≥ 2: Heterogeneous**  
  A score this high indicates significant differences between sites or regions, suggesting that the regional model might not be appropriate for all the data. The region may be too diverse or require model adjustments.

- **Z-Score (Goodness of Fit)**

  - **Low Z-Score**: A low Z-Score suggests a good fit between the model and the observed data. Typically, an absolute Z-Score below 1.64 indicates that the model is appropriate and the fit is statistically acceptable.

  - **High Z-Score**: A high Z-Score suggests significant discrepancies between observed and expected values. An absolute Z-Score above 1.64 implies that the model might not fit the data well, and further adjustments or a different model might be needed.

The function `xhydro.frequency_analysis.regional.calc_h_z` is used to compute the *H-Score* and *Z-Score* for the regional frequency analysis.

<div class="alert alert-warning"><b>WARNING</b>
    
  The `kap` argument in the `xhydro.frequency_analysis.regional.calc_h_z` function expects a Kappa3 distribution object generated using `lmoments3.distr.KappaGen()`. However, due to licensing issues, the `lmoments3` library cannot currently be included in the `xHydro` requirements and must be installed independently.
  
</div>



In [ ]:
help(xhfa.regional.calc_h_z)

In [ ]:
ds_hz = xhfa.regional.calc_h_z(ds_groups, ds_moments_groups, kap=KappaGen())
ds_hz

We can filter the results to include only the data that has H and Z scores below given thresholds using the function `xhydro.frequency_analysis.regional.mask_h_z`. By default, the thresholds are set to 1 for H and 1.64 for Z. However, these thresholds can be customized to suit the specific requirements of the analysis.


In [ ]:
help(xhfa.regional.mask_h_z)

In [ ]:
mask = xhfa.regional.mask_h_z(ds_hz)
ds_groups_h1 = ds_groups.where(mask).load()
ds_moments_groups_h1 = ds_moments_groups.where(mask).load()

The final step of the regional frequency analysis is similar to the local frequency analysis, but instead of using `xhydro.frequency_analysis.local.parametric_quantiles`, we use `xhydro.frequency_analysis.regional.calculate_rp_from_afr` to calculate the return periods. Additionally, to avoid performing the analysis on very small regions, the `remove_small_regions` argument can be used to exclude any regions with less than a specified number of stations. By default, this threshold is set to 5.


In [ ]:
help(xhfa.regional.calculate_rp_from_afr)

In [ ]:
help(xhfa.regional.remove_small_regions)

In [ ]:
q_t = xhfa.regional.calculate_rp_from_afr(
    ds_groups_h1, ds_moments_groups_h1, rp=[2, 20, 100]
)
q_t

In [ ]:
q_t = xhfa.regional.remove_small_regions(q_t)
q_t

Let's plot results for one station, comparing local and regional analyses.

In [ ]:
q_reg = q_t.sel(id="023401").dropna(dim="group_id", how="all")
reg = q_reg.q_max_annual.squeeze()

In [ ]:
params_loc = xhfa.local.fit(ds_4fa.sel(id="023401"))
q_loc = xhfa.local.parametric_quantiles(params_loc, t=[2, 20, 100])
loc = q_loc.sel(scipy_dist="genextreme").q_max_annual

In [ ]:
fig = plt.figure(figsize=(15, 4))
plt.plot(reg.return_period.values, reg.values, "red", label="Regional analysis")
plt.plot(loc.return_period.values, loc.values, "black", label="Local analysis")
plt.xscale("log")
plt.grid(visible=True)
plt.xlabel("Return period (years)")
plt.ylabel("Streamflow (m$^3$/s)")
plt.legend()

## Uncertainties

Uncertainties are an important aspect of frequency analysis and should be considered when interpreting results. These uncertainties often stem from data quality, the choice of distribution, and the estimation of parameters. While visualizations can provide insights into the model fit, it’s crucial to quantify and account for uncertainties, such as confidence intervals for parameter estimates, to ensure robust conclusions.

### a) Bootstrapping the observations

One method for quantifying uncertainties is to bootstrap the observations. In this example, we will perform bootstrapping a small number of times to illustrate the process, though in practice, a higher number of iterations (e.g., 5000) is recommended to obtain more reliable estimates. Bootstrapping resamples the observed data with replacement to generate multiple datasets, which can then be used to assess the variability in the model's parameters and results.

This can be accomplished by calling `xhydro.frequency_analysis.uncertainties.bootstrap_obs`. One key difference with local analyses is that for regional analyses, no fitting step is involved, making this function significantly faster. Instead, the results are obtained by first using `xhydro.frequency_analysis.uncertainties.calc_moments_iter` to compute L-moments for all bootstrap samples. Then, `xhydro.frequency_analysis.uncertainties.calc_q_iter` is used to calculate the return periods based on those L-moments.

In [ ]:
help(xhfa.uncertainties.calc_moments_iter)

In [ ]:
help(xhfa.uncertainties.calc_q_iter)

In [ ]:
ds_reg_boot = xhfa.uncertainties.bootstrap_obs(ds_4fa, n_samples=35)
ds_moments_iter = xhfa.uncertainties.calc_moments_iter(ds_reg_boot).load()
ds_moments_iter

In [ ]:
q_reg_boot = xhfa.uncertainties.calc_q_iter(
    bv="023401",
    var="q_max_annual",
    ds_groups=ds_groups_h1,
    ds_moments_iter=ds_moments_iter,
    return_periods=[2, 20, 100],
)
q_reg_boot

### b) Using multiple regions

Another approach to estimating uncertainty is by considering multiple regions for a single catchment of interest. This can be done by applying different clustering methods or by performing a jackknife procedure on the station list. While this process can be computationally intensive, we present a simplified example here to illustrate the potential of this method. The main objective is to demonstrate how considering multiple regions can help in evaluating uncertainty in the regional frequency analysis.

First, we will use `xhydro.frequency_analysis.uncertainties.generate_combinations` on the results from our PCA to generate lists of stations. This function creates combinations that exclude a certain number of stations each time, allowing us to evaluate how the exclusion of different stations impacts the groupings.


In [ ]:
help(xhfa.uncertainties.generate_combinations)

In [ ]:
combinations_list = xhfa.uncertainties.generate_combinations(data_pca, n=2)
print(
    f"This generated {len(combinations_list)} lists of stations, varying from {len(combinations_list[0])} to {len(combinations_list[-1])} stations."
)

Next, we will combine those lists with three clustering methods and, for each method, we'll try to change some of the parameters.


In [ ]:
clust_options = {
    AgglomerativeClustering: {"arg_name": "n_clusters", "range": range(2, 12)},
    HDBSCAN: {"arg_name": "min_cluster_size", "range": range(6, 7)},
    OPTICS: {"arg_name": "min_samples", "range": range(4, 5)},
}

In [ ]:
groups = []

for model in [AgglomerativeClustering, HDBSCAN, OPTICS]:

    for p in clust_options[model]["range"]:
        d_param = {}
        d_param[clust_options[model]["arg_name"]] = p
        for combination in combinations_list:
            # Extract data for the current combination
            data_com = data_pca.sel(Station=list(combination))
            # Get groups from the fit and add to the list
            groups = groups + xhfa.regional.get_group_from_fit(model, d_param, data_com)
unique_groups = [list(x) for x in {tuple(x) for x in groups}]

The next steps follow a similar approach to the previous ones, but now with multiple regions.

In [ ]:
ds_groups_multiregion = xhfa.regional.group_ds(ds_4fa, groups=unique_groups)
ds_moments_groups_multiregion = xhfa.regional.group_ds(ds_moments, groups=unique_groups)
ds_h_z_multiregion = xhfa.regional.calc_h_z(
    ds_groups_multiregion, ds_moments_groups_multiregion, kap=KappaGen()
)
mask_multiregion = xhfa.regional.mask_h_z(ds_h_z_multiregion)
ds_groups_h1_multiregion = ds_groups_multiregion.where(mask_multiregion).load()
ds_moments_groups_h1_multiregion = ds_moments_groups_multiregion.where(
    mask_multiregion
).load()

q_t_multiregion = xhfa.regional.calculate_rp_from_afr(
    ds_groups_h1_multiregion, ds_moments_groups_h1_multiregion, rp=[2, 20, 100]
)
q_t_multiregion = xhfa.regional.remove_small_regions(q_t_multiregion)

q = q_t_multiregion.sel(id="023401").dropna(dim="group_id", how="all")
q

### c) Combining the bootstrap with multiple regions

It is possible to combine both methods—bootstrapping and multiple regions—by integrating the following:

1. Bootstrapped L-moments from `xhydro.frequency_analysis.uncertainties.calc_moments_iter`.
2. Masked results from `xhydro.frequency_analysis.regional.group_ds` that have been filtered based on the H and Z-Scores.

When combining these methods, `xhydro.frequency_analysis.uncertainties.calc_q_iter` will check in how many distinct `group_id` values the station is present and will stack the data accordingly with the bootstrap samples. For instance, if we have 35 bootstraps and roughly 220 regions, this will generate a total of around 7,700 samples for further analysis.


In [ ]:
q_reg_multiregion_boot = xhfa.uncertainties.calc_q_iter(
    "023401",
    "q_max_annual",
    ds_groups_h1_multiregion,
    ds_moments_iter,
    return_periods=[2, 20, 100],
)
q_reg_multiregion_boot

### d) Comparison

Let's show the difference between approaches.

In [ ]:
fig, ax = plt.subplots(1, 3)
fig.set_figheight(4)
fig.set_figwidth(15)

# Subset the data
q_reg_boots = q_reg_boot.q_max_annual.sel(id="023401")
q_t_multiregions = q_t_multiregion.q_max_annual.sel(id="023401")
q_t_multiregion_boots = q_reg_multiregion_boot.q_max_annual.sel(id="023401")


def _make_plot(data, dim, label):
    # Original fit
    ax.plot(
        loc.return_period.values,
        loc,
        "black",
        label="Local frequency analysis",
    )
    ax.plot(
        reg.return_period.values,
        reg,
        "red",
        label="Original regional frequency analysis",
    )

    ax.plot(
        data.return_period.values,
        data.quantile(0.5, dim),
        "green",
        label=label,
    )
    data_05 = data.quantile(0.05, dim)
    data_95 = data.quantile(0.95, dim)
    ax.fill_between(
        data.return_period.values, data_05, data_95, alpha=0.2, color="green"
    )

    plt.xscale("log")
    plt.grid(visible=True)
    plt.xlabel("Return period (years)")
    plt.ylabel("Streamflow (m$^3$/s)")
    plt.ylim([150, 500])
    ax.legend()


ax = plt.subplot(1, 3, 1)
_make_plot(q_reg_boots, "samples", "Bootstrapped observations")
ax = plt.subplot(1, 3, 2)
_make_plot(q_t_multiregions, "group_id", "Multiple regions")
ax = plt.subplot(1, 3, 3)
_make_plot(q_t_multiregion_boots, "samples", "Combined")